Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import libraries

In [2]:
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import matplotlib.style as ms
ms.use('seaborn-muted')
%matplotlib nbagg
%matplotlib inline
import IPython.display as ipd
from IPython.display import Audio
import os, sys
from scipy.io import wavfile
from sklearn.preprocessing import LabelEncoder, scale
import time
import tensorflow as tf
tf.config.list_physical_devices('GPU')
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
from numpy import asarray
from numpy import savetxt
from tensorflow.keras.callbacks import ReduceLROnPlateau

import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.utils import to_categorical
from keras.utils import np_utils
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, Input, MaxPooling2D, Conv1D, MaxPooling1D, Lambda
from tensorflow.keras import losses
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5



In [3]:
!pip install pydub &> /dev/null
!pip install sounddevice &> /dev/null
!pip install soundfile &> /dev/null
!sudo apt-get install libportaudio2 &> /dev/null
!pip install pysndfx &> /dev/null
!pip install python_speech_features &> /dev/null
#!sudo apt-get install sox &> /dev/null

Load Weights

In [108]:
from keras.models import load_model
model = load_model('/content/drive/MyDrive/SPEECH RECOGNITION FINAL/WEIGHTS/weights_of_audio.best.hdf5')

labels = ['eight', 'sheila', 'nine', 'yes', 'one', 'no', 'left', 'tree', 'bed', 'bird', 'go', 'wow', 'seven', 'marvin', 'dog', 'three', 'two', 'house', 'down', 'six', 'five', 'off', 'right', 'cat', 'zero', 'four', 'stop', 'up', 'on', 'happy']

all_labels = np.load(os.path.join("/content/drive/MyDrive/SPEECH RECOGNITION FINAL/ALL_LABELS/all_labels.npy"))

le = LabelEncoder()
y=le.fit_transform(all_labels)
classes= list(le.classes_)

y = np_utils.to_categorical(y, num_classes=len(labels))

print(y.shape)

#np.save(os.path.join(path2, 'all_labels_encoded'), y)

#Predict probabilities for all speeches. Maximum probabilty with a threshold of 0.6 is chosen as predicted speech.

from datetime import datetime 
import pytz 
import csv
import random
  
# get the standard UTC time  
UTC = pytz.utc 
IST = pytz.timezone('Asia/Kolkata') 

def predict(audio,n, k=0.45):
    prob=model.predict_proba(audio.reshape(1,8000,1))
    index=np.argmax(prob[0])
    #print(prob[0])  #Print all probabilities
    #print(max(prob[0]))     #Print max probability
    if max(prob[0]) > k:
        #print(index)
        f=open(r'/content/drive/MyDrive/SPEECH RECOGNITION FINAL/Excel Files/Texter' + '{0}'.format(n)+'.txt', 'a')
        f.write(str(datetime.now(IST)) + "," + classes[index] + "\n")
        f.close()
        return classes[index]

# Import the AudioSegment class for processing audio and the 
# split_on_silence function for separating out silent chunks.
from pydub import AudioSegment
from pydub.silence import split_on_silence

# Define a function to normalize a chunk to a target amplitude.
def match_target_amplitude(aChunk, target_dBFS):
    ''' Normalize given audio chunk '''
    change_in_dBFS = target_dBFS - aChunk.dBFS
    return aChunk.apply_gain(change_in_dBFS)

'''from pathlib import Path
AudioSegment.converter = r"H:\\whizkey\\FINAL PAPER\\ffmpeg\\ffmpeg.exe"
AudioSegment.ffprobe   = r"H:\\whizkey\\FINAL PAPER\\ffmpeg\\ffprobe.exe"
 
print (AudioSegment.converter)
print (AudioSegment.ffprobe)'''
 
#my_file = Path(r"H:\\whizkey\\FINAL PAPER\\cont speech\\newer10.mp3")
#print ('ID1 : %s' % my_file) 
#audio = AudioSegment.from_file_using_temporary_files(my_file)    

from os import path
from pydub import AudioSegment

def Speech_recognition(src1,dst1,min_silence_len = 200):
    # files                                                                         
    src = r"/content/drive/MyDrive/SPEECH RECOGNITION FINAL/Continuous Speech/" + src1 + ".mp3"
    dst = r"/content/drive/MyDrive/SPEECH RECOGNITION FINAL/Continuous Speech/" + dst1 + ".wav"

    # convert wav to mp3                                                            
    song1 = AudioSegment.from_file_using_temporary_files(src)
    song1.export(dst, format="wav")

#Calculate dBFS (Decibels relative to full scale)

    dBFS = song1.dBFS
    chunks = split_on_silence(
        song1,
        min_silence_len = 200,          #Set minimum silence length threshold to 200 ms
        silence_thresh = dBFS-16
    )

#Split the continuous audio into individual speech commands.

    # Process each chunk with your parameters
    n = random. randint(0,9999999999999999999999) 

    for i, chunk in enumerate(chunks):
        # Create a silence chunk that's 0.5 seconds (or 500 ms) long for padding.
        silence_chunk = AudioSegment.silent(duration=200)

        # Add the padding chunk to beginning and end of the entire chunk.
        audio_chunk = silence_chunk + chunk + silence_chunk

        # Normalize the entire chunk.
        normalized_chunk = match_target_amplitude(audio_chunk, -20.0)

        # Export the audio chunk with new bitrate.
        print("Exporting chunker{0}.wav.".format(i))
        k = normalized_chunk.export(
            r'/content/drive/MyDrive/SPEECH RECOGNITION FINAL/Continuous Speech/Chunker/chunker{0}.wav' .format(i),
            bitrate = "192k",
            format = "wav"
        )

#Resample each speech command to 8000 Hz
        
    time_start = time.time()
    ctr = 0
    for i, chunk in enumerate(chunks):
        ctr += 1
        melspec1 = []
        filepath=r'/content/drive/MyDrive/SPEECH RECOGNITION FINAL/Continuous Speech/Chunker/chunker{0}.wav'.format(i)
        print(filepath)
        
        samples, sample_rate = librosa.load(filepath, sr = 16000)
        samples = librosa.resample(samples, 16000, 8000)
        samples.resize(8000)

        pred = predict(samples,n)         #Prediction after resampling to 8000Hz    
        print("Text:",pred)

    print('Run time: {}'.format(time.time()-time_start))
    print('Total speech commands: ',ctr)
    dataframe1 = pd.read_csv(r"/content/drive/MyDrive/SPEECH RECOGNITION FINAL/Excel Files/Texter{0}.txt".format(n),header=None) 
    print('Created txt')
    dataframe1.to_csv(r"/content/drive/MyDrive/SPEECH RECOGNITION FINAL/Excel Files/Texter{0}.csv".format(n),index = None)
    print("/content/drive/MyDrive/SPEECH RECOGNITION FINAL/Excel Files/Texter{0}.csv".format(n))

(58252, 30)


In [110]:
file_name = 'newer'
file_no = str(input("Enter File No\n"))
file_name = file_name + file_no
Speech_recognition(file_name,file_name)

Enter File No
35
Exporting chunker0.wav.
Exporting chunker1.wav.
Exporting chunker2.wav.
Exporting chunker3.wav.
Exporting chunker4.wav.
Exporting chunker5.wav.
Exporting chunker6.wav.
Exporting chunker7.wav.
/content/drive/MyDrive/SPEECH RECOGNITION FINAL/Continuous Speech/Chunker/chunker0.wav
Text: three
/content/drive/MyDrive/SPEECH RECOGNITION FINAL/Continuous Speech/Chunker/chunker1.wav
Text: seven
/content/drive/MyDrive/SPEECH RECOGNITION FINAL/Continuous Speech/Chunker/chunker2.wav
Text: one
/content/drive/MyDrive/SPEECH RECOGNITION FINAL/Continuous Speech/Chunker/chunker3.wav
Text: five
/content/drive/MyDrive/SPEECH RECOGNITION FINAL/Continuous Speech/Chunker/chunker4.wav
Text: eight
/content/drive/MyDrive/SPEECH RECOGNITION FINAL/Continuous Speech/Chunker/chunker5.wav
Text: nine
/content/drive/MyDrive/SPEECH RECOGNITION FINAL/Continuous Speech/Chunker/chunker6.wav
Text: zero
/content/drive/MyDrive/SPEECH RECOGNITION FINAL/Continuous Speech/Chunker/chunker7.wav
Text: zero
Run t

In [121]:
ipd.Audio('/content/drive/MyDrive/SPEECH RECOGNITION FINAL/Continuous Speech/Chunker/chunker0.wav', rate=8000)